### Deadline: 4th December, 2024 at 23:59 CET

### Name & Student No.:
- Abdel Kader, Schehat & 10064822
- Tarbouch, Johny & 10033994
- Asfari, Mustafa & 10075209

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

## Load the data and explore it


In [2]:
train = torch.load("/kaggle/input/har-data/HAR/train.pt")
val = torch.load("/kaggle/input/har-data/HAR/val.pt")
test = torch.load("/kaggle/input/har-data/HAR/test.pt")

# Remove the last 78 time steps in the feature vector as described in the forum from Sikdar
train['samples'] = train['samples'][:, :, :128]
val['samples'] = val['samples'][:, :, :128]
test['samples'] = test['samples'][:, :, :128]

/tmp/ipykernel_138/474991073.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train = torch.load("/kaggle/input/har-data/HAR/train.pt")
/tmp/ipykernel_138/474991073.py:2:

In [3]:
train

{'samples': tensor([[[ 1.9044e-01,  1.9105e-01,  1.8998e-01,  ...,  1.8287e-01,
            1.7958e-01,  1.8497e-01],
          [ 7.8786e-01,  7.8715e-01,  7.8758e-01,  ...,  7.8876e-01,
            7.8801e-01,  7.8668e-01],
          [ 5.7340e-01,  5.6987e-01,  5.7117e-01,  ...,  5.7183e-01,
            5.7341e-01,  5.7242e-01]],
 
         [[ 6.1260e-01,  6.6497e-01,  7.8970e-01,  ...,  7.4186e-01,
            8.2214e-01,  9.5540e-01],
          [-2.8010e-01, -3.0221e-01, -3.0298e-01,  ..., -6.1126e-02,
           -3.2162e-02, -2.2901e-02],
          [-2.6934e-01, -3.2591e-01, -3.4449e-01,  ..., -2.7576e-01,
           -2.4760e-01, -2.2416e-01]],
 
         [[ 1.0204e+00,  1.0297e+00,  1.0322e+00,  ...,  7.6671e-01,
            6.9077e-01,  7.3700e-01],
          [-8.0171e-02, -9.7238e-02, -1.3547e-01,  ..., -2.7247e-01,
           -3.4729e-01, -3.6471e-01],
          [-7.0868e-02, -7.6643e-02, -9.4666e-02,  ..., -3.0539e-02,
           -8.8336e-04,  2.3590e-02]],
 
         ...,
 
 

In [4]:
type(train)

dict

In [5]:
len(train)

2

**The data is in the dimension form (samples, sequence, feature)**
- In training set:
- 5881 samples
- 3 time steps forming a sequence
- 128 length of feature vector

In [6]:
print(f"Train data samples shape: {train['samples'].shape}")
print(f"Validation data samples shape: {val['samples'].shape}")
print(f"Test data samples shape: {test['samples'].shape}")

Train data samples shape: torch.Size([5881, 3, 128])
Validation data samples shape: torch.Size([1471, 3, 128])
Test data samples shape: torch.Size([2947, 3, 128])


In [7]:
print(f"Train data label shape: {train['labels'].shape}")
print(f"Validation label data shape: {val['labels'].shape}")
print(f"Test data label shape: {test['labels'].shape}")

Train data label shape: torch.Size([5881])
Validation label data shape: torch.Size([1471])
Test data label shape: torch.Size([2947])


In [8]:
train["labels"].unique()

tensor([0., 1., 2., 3., 4., 5.], dtype=torch.float64)

In [9]:
train["samples"][0]

tensor([[0.1904, 0.1911, 0.1900, 0.1898, 0.1899, 0.1883, 0.1886, 0.1879, 0.1831,
         0.1819, 0.1864, 0.1916, 0.1912, 0.1899, 0.1929, 0.1927, 0.1926, 0.1940,
         0.1917, 0.1890, 0.1841, 0.1821, 0.1870, 0.1884, 0.1876, 0.1893, 0.1908,
         0.1905, 0.1888, 0.1897, 0.1935, 0.1961, 0.1918, 0.1843, 0.1846, 0.1853,
         0.1819, 0.1827, 0.1860, 0.1913, 0.1942, 0.1923, 0.1934, 0.1943, 0.1940,
         0.1936, 0.1889, 0.1845, 0.1824, 0.1830, 0.1842, 0.1835, 0.1871, 0.1911,
         0.1895, 0.1881, 0.1880, 0.1873, 0.1863, 0.1860, 0.1863, 0.1869, 0.1879,
         0.1875, 0.1873, 0.1878, 0.1873, 0.1875, 0.1877, 0.1883, 0.1893, 0.1885,
         0.1866, 0.1839, 0.1829, 0.1855, 0.1880, 0.1888, 0.1899, 0.1907, 0.1902,
         0.1898, 0.1881, 0.1842, 0.1823, 0.1835, 0.1838, 0.1842, 0.1871, 0.1893,
         0.1899, 0.1905, 0.1902, 0.1890, 0.1879, 0.1871, 0.1875, 0.1877, 0.1873,
         0.1878, 0.1884, 0.1890, 0.1891, 0.1886, 0.1890, 0.1890, 0.1891, 0.1908,
         0.1916, 0.1888, 0.1

## Dataset Class

In [10]:
class HARDataset(Dataset):
    def __init__(self, samples, labels):
        self.samples = samples
        self.labels = labels.long()

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        label = self.labels[idx]
        return sample, label

## Model

In [11]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.rnn(x, h0)
        # -1 to take last time step
        out = self.fc(out[:, -1, :])
        return out

input_size = train['samples'].shape[2]
hidden_size = 256
output_size = len(train['labels'].unique())
num_layers = 1

model = RNNModel(input_size, hidden_size, output_size, num_layers)
print(model)

RNNModel(
  (rnn): RNN(128, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=6, bias=True)
)


## Training

In [12]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create data loaders
train_dataset = HARDataset(train['samples'], train['labels'])
val_dataset = HARDataset(val['samples'], val['labels'])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Training loop
num_epochs = 20
best_val_accuracy = 0.0
best_model = None

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    # Batch training
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_predictions = []
    val_targets = []
    
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            outputs = model(batch_x)
            _, predicted = torch.max(outputs.data, 1)
            val_predictions.extend(predicted)
            val_targets.extend(batch_y)
    
    val_accuracy = accuracy_score(val_targets, val_predictions)
    avg_train_loss = train_loss / len(train_loader)
    
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = model.state_dict()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Accuracy: {val_accuracy:.4f}")

model.load_state_dict(best_model)

Epoch [1/20], Loss: 0.9820, Validation Accuracy: 0.5364
Epoch [2/20], Loss: 1.1163, Validation Accuracy: 0.5867
Epoch [3/20], Loss: 0.8124, Validation Accuracy: 0.5799
Epoch [4/20], Loss: 0.7354, Validation Accuracy: 0.7267
Epoch [5/20], Loss: 0.4618, Validation Accuracy: 0.8090
Epoch [6/20], Loss: 0.3248, Validation Accuracy: 0.8450
Epoch [7/20], Loss: 0.1252, Validation Accuracy: 0.8749
Epoch [8/20], Loss: 0.4736, Validation Accuracy: 0.9109
Epoch [9/20], Loss: 0.1696, Validation Accuracy: 0.9205
Epoch [10/20], Loss: 0.1149, Validation Accuracy: 0.9137
Epoch [11/20], Loss: 0.2224, Validation Accuracy: 0.9218
Epoch [12/20], Loss: 0.2791, Validation Accuracy: 0.9239
Epoch [13/20], Loss: 0.0495, Validation Accuracy: 0.9252
Epoch [14/20], Loss: 0.0773, Validation Accuracy: 0.9109
Epoch [15/20], Loss: 0.1485, Validation Accuracy: 0.9075
Epoch [16/20], Loss: 0.1826, Validation Accuracy: 0.9409
Epoch [17/20], Loss: 0.0542, Validation Accuracy: 0.9422
Epoch [18/20], Loss: 0.0503, Validation 

<All keys matched successfully>

## Evaluation

In [13]:
# Test accuracy
def evaluate_model(model, test_data):
    model.eval()
    with torch.no_grad():
        samples = test_data['samples']
        labels = test_data['labels']
        
        outputs = model(samples)
        _, predicted = torch.max(outputs.data, 1)
        
        accuracy = accuracy_score(labels, predicted)
                
    return accuracy

test_accuracy = evaluate_model(model, test)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.8405
